In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
from statsmodels.tsa.api import ExponentialSmoothing , SimpleExpSmoothing , Holt
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [15]:
data = pd.read_csv("./data/gold_monthly.csv")

In [16]:
data.head

,Date,Price
0,1950-01,34.73
1,1950-02,34.73
2,1950-03,34.73
3,1950-04,34.73
4,1950-05,34.73
